In [1]:
! unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
! unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
! unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip

Archive:  /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    
Archive:  /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
  inflating: testData.tsv            
Archive:  /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  


Part 2中训练好的word2vec模型，对于每一个词都有一个特征向量，用numpy存储，可以通过syn0调用

In [2]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("/kaggle/input/inputdata/300features_40minwords_10context")

In [3]:
model.wv.vectors.shape

(16490, 300)

syn0的每一行，即代表词汇表中的一个单词，即有16490个单词。列代表特征向量的大小，即300，这个是我们在part 2训练时设定的数字。我们设置的最小单词频度是40（即出现40次以下的单词会被忽略），最后得到一个有16492个单词的词汇表，每个词有300个特征。

单个词向量可以通过下面的方法查看

In [4]:
model.wv['flower'][:20]

array([-0.23710842,  0.08830173, -0.03082926,  0.1317306 ,  0.01820597,
        0.25649256,  0.12806156,  0.20457655,  0.28028506, -0.51132834,
       -0.09054958,  0.11458612,  0.07447369,  0.33142266, -0.30471832,
       -0.35287043, -0.18231627, -0.4457859 , -0.02176598,  0.31776565],
      dtype=float32)

这里导入之前的数据集：

In [5]:
import pandas as pd

train = pd.read_csv("/kaggle/working/labeledTrainData.tsv", header=0, 
                     delimiter="\t", quoting=3)

test = pd.read_csv( "/kaggle/working/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv("/kaggle/working/unlabeledTrainData.tsv", header=0, 
                              delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" 
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



导入之前的文本预处理函数：

In [6]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # 函数用于将文档转换成单词序列，可选择是否移除停用词。返回一个单词列表。
    
    # 1. 移除HTML标签
    review_text = BeautifulSoup(review, 'lxml').get_text()
      
    # 2. 移除非字母字符
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    
    # 3. 将所有单词转换为小写并分割它们
    words = review_text.lower().split()

    # 4. 可选择是否移除停用词（默认为False）
    if remove_stopwords:
        stops = set(stopwords.words("english"))  # 从nltk库中加载英文停用词列表，并转换为集合
        words = [w for w in words if not w in stops]  # 移除停用词
    
    # 5. 返回单词列表
    return words

导入之前的函数：使用NLTK库的punkt分词器将一段评论文本分割成句子，并将每个句子进一步处理成单词列表。

In [9]:
import nltk.data
# nltk.download()   

# 加载punkt分词器
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# 定义一个函数，将评论分割成解析后的句子
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # 这个函数将评论分割成解析后的句子。返回一个句子列表，其中每个句子是一个单词列表
    
    # 1. 使用NLTK分词器将段落分割成句子
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. 循环遍历每个句子
    sentences = []
    for raw_sentence in raw_sentences:
        # 如果一个句子为空，跳过它
        if len(raw_sentence) > 0:
            # 否则，调用review_to_wordlist函数来获取单词列表
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))

    # 返回句子列表（每个句子是一个单词列表，
    # 因此，这返回一个列表的列表
    return sentences

**由单词到段落，尝试1：向量平均**

电影评论数据集处理起来一个比较麻烦的地方在于，评论的长度是不一样的。我们需要提取出每一个词的向量，然后把它们转换为一个特征集，而且每个评论的特征长度是一样的。


因为每一个单词有一个300维的特征，我们可以用特征操作来把一个评论中的单词合并起来。一个简单的方法就是对所有的词向量取平均。（如果取平均的话，我们需要移除stop words，因为会带来噪音）


下面是计算特征向量平均值的代码：

In [10]:
import numpy as np 

def makeFeatureVec(words, model, num_features):
    # 函数用于平均给定段落中的所有词向量
    # 预初始化一个空的numpy数组（为了速度）
    featureVec = np.zeros((num_features,), dtype="float32")
    
    nwords = 0
    
    # index2word是包含模型词汇表中单词名称的列表。将其转换为集合，以提高速度
    index2word_set = set(model.wv.index_to_key)
    
    # 循环遍历评论中的每个词，如果它在模型的词汇表中，
    # 将其特征向量加到总数中
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            featureVec = np.add(featureVec, model.wv[word])
    
    # 将结果除以单词数量以得到平均值
    featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # 给定一组评论（每个评论是一个单词列表），计算每个评论的平均特征向量并返回一个2D numpy数组 
    
    # 初始化一个计数器
    counter = 0
    
    # 预分配一个2D numpy数组，为了速度
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # 循环遍历评论
    for review in reviews:
       
        # 每处理1000个评论打印一次状态消息
        if counter%1000 == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
       
        # 调用上面定义的函数，生成平均特征向量
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       
        # 增加计数器
        counter = counter + 1
    return reviewFeatureVecs

调用上面的函数来给每一个评论创建一个平均向量。

In [11]:
# 设置各种参数的值
num_features = 300    # 词向量的维度
min_word_count = 40   # 最低词频
num_workers = 4       # 并行运行的线程数量
context = 10          # 上下文窗口大小
downsampling = 1e-3   # 频繁词的降采样设置

In [12]:
# ****************************************************************
# 计算训练集的平均特征向量

clean_train_reviews = []
for review in train["review"]:
    # 对每个训练集评论调用review_to_wordlist函数，并移除停用词
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
# 输出消息，表示正在为测试集评论创建平均特征向量
print("Creating average feature vecs for test reviews")

clean_test_reviews = []
for review in test["review"]:
    # 对每个测试集评论调用review_to_wordlist函数，并移除停用词
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

/tmp/ipykernel_31/592310003.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, 'lxml').get_text()


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

使用随机森林分类器对训练数据进行拟合，并在测试数据上进行预测，最后将预测结果保存

In [13]:
# 初始化一个随机森林分类器，使用100棵树
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
# 使用训练数据的特征向量和情感标签来拟合随机森林模型
forest = forest.fit( trainDataVecs, train["sentiment"] )

# 在测试数据上进行预测并提取结果
result = forest.predict( testDataVecs )

# 将测试数据的id和预测结果组合成一个新的DataFrame
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "/kaggle/working/submission.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
